In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead, LineByLineTextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, TrainerCallback,ProgressCallback
import torch
import random

In [2]:
# Carregar o modelo e o tokenizador
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Definir o comprimento máximo de sequência para 1024 tokens
tokenizer.model_max_length = 1024


/home/rafaelrosendo/anaconda3/envs/torch2/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
# Carregar seus dados do CSV (substitua o valor de 'path_to_csv' pelo caminho do seu arquivo CSV)
path_to_csv = "/home/rafaelrosendo/gpt2/portuguese-poems.csv"
data = pd.read_csv(path_to_csv)["Content"].tolist()

In [4]:
# Embaralhar a lista de dados para garantir uma divisão aleatória
random.shuffle(data)

# Definir a proporção do conjunto de validação (20%)
validation_ratio = 0.2

# Calcular o índice para dividir os dados
split_index = int(len(data) * (1 - validation_ratio))

# Dividir os dados em treinamento e validação
train_data = data[:split_index]
validation_data = data[split_index:]

In [5]:
# Converter os dados em um formato adequado para o treinamento do modelo
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_to_csv,
    block_size=128,
)
validation_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_to_csv,
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



/home/rafaelrosendo/anaconda3/envs/torch2/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./gpt2_fine_tuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [10]:
# Inicializar o Trainer e iniciar o treinamento
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    callbacks=[ProgressCallback()]
)

trainer.train()

You are adding a <class 'transformers.trainer_callback.ProgressCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
ProgressCallback
/home/rafaelrosendo/anaconda3/envs/torch2/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/402460 [00:00<?, ?it/s]

  0%|          | 0/402460 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 5.8952, 'learning_rate': 4.9937882025542915e-05, 'epoch': 0.01}
{'loss': 5.8952, 'learning_rate': 4.9937882025542915e-05, 'epoch': 0.01}
{'loss': 5.7537, 'learning_rate': 4.987576405108583e-05, 'epoch': 0.01}
{'loss': 5.7537, 'learning_rate': 4.987576405108583e-05, 'epoch': 0.01}
{'loss': 5.7623, 'learning_rate': 4.981364607662873e-05, 'epoch': 0.02}
{'loss': 5.7623, 'learning_rate': 4.981364607662873e-05, 'epoch': 0.02}
{'loss': 5.6725, 'learning_rate': 4.9751528102171645e-05, 'epoch': 0.02}
{'loss': 5.6725, 'learning_rate': 4.9751528102171645e-05, 'epoch': 0.02}
{'loss': 5.647, 'learning_rate': 4.968941012771456e-05, 'epoch': 0.03}
{'loss': 5.647, 'learning_rate': 4.968941012771456e-05, 'epoch': 0.03}
{'loss': 5.6499, 'learning_rate': 4.962729215325747e-05, 'epoch': 0.04}
{'loss': 5.6499, 'learning_rate': 4.962729215325747e-05, 'epoch': 0.04}
{'loss': 5.5911, 'learning_rate': 4.9565174178800375e-05, 'epoch': 0.04}
{'loss': 5.5911, 'learning_rate': 4.9565174178800375e-05, 'ep